# Лабораторная работа №4
## Выполнил студент группы БВТ1905 Баев Дмитрий

Реализовать следующие структуры данных:
* Стек (stack):
    * операции для стека: 
        * инициализация;
        * проверка на пустоту;
        * добавление нового элемента в начало;
        * извлечение элемента из начала;
* Дек (двусторонняя очередь, deque):
    * операции для дека: 
        * инициализация;
        * проверка на пустоту;
        * добавление нового элемента в начало; 
        * добавление нового элемента в конец;
        * извлечение элемента из начала; 
        * извлечение элемента из конца;

Разработать программу обработки данных, содержащихся в заранее
подготовленном txt-файле, в соответствии с заданиями, применив
указанную в задании структуру данных. Результат работы программы
вывести на экран и сохранить в отдельном txt-файле.


In [177]:
# Библиотека для копирования листов, используется в Деке
import copy

In [178]:
class Stack:
    # Инициализатор
    def __init__(self, size):
        # Создаем массив указаного размера
        self.arr = ['empty'] * size
        # Кол-во занятых переменных
        self.count = 0
        
    # Добавление нового элемента в начало
    def push(self, elem):
        # Если проверка на переполнения не пройдена, возникает ошибка
        if self.count + 1 > len(self.arr): 
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        self.arr[self.count] = elem
        self.count += 1
        
    # Извлечение элемента с начала
    def pop(self):
        if self.count - 1 >= 0:
            elem = self.arr[self.count-1]
            self.arr[self.count-1] = 'empty'
            self.count -= 1
            if elem == None:
                return 0
            else:
                return elem
            
    def top(self):
        elem = self.arr[self.count-1]
        if elem == None:
            return 0
        else:
            return elem
            
    # Проверка на пустоту        
    def isEmpty(self):
        if self.count == 0:
                return True        
        return False
    
    # Вывод содержимого стека
    def print(self):
        print("Stack =", self.arr,"len =", self.count)

In [179]:

class Deque(Stack):
    # Инициализатор
    def __init__(self, size):
        self.j = -1
        Stack.__init__(self, size)
    
    # Сдвигаем все элементы массива в нужную сторону
    def _move(self, direct):
        if direct == 'right' and self.arr[len(self.arr) - 1] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = 0
            for i in range(len(self.arr)):
                if i >= len(self.arr) - self.count:
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
        if direct == 'left' and self.arr[0] == 'empty':
            nArr = ['empty'] * len(self.arr)
            j = len(self.arr) - self.count
            for i in range(len(self.arr)):
                if j < len(self.arr):
                    nArr[i] = self.arr[j]
                    j += 1
            self.arr = nArr.copy()
            
    # Добавление нового элемента в начало
    def push(self, elem):
        Deque._move(self, 'left')
        Stack.push(self, elem)
        
    # Извлечение элемента с начала    
    def pop(self):
        Deque._move(self, 'left')
        return Stack.pop(self)
    
    # Добавление нового элемента в конец
    def pushEnd(self, elem):
        if len(self.arr) < 2:
            Stack.push(self, elem)
        if self.count + 1 > len(self.arr):
            raise Exception('Элемент {}, не может быть помещен в Stack, так как он будет переполнен.'.format(elem))
        else:
            Deque._move(self, 'right')
            self.arr[len(self.arr) - self.count - 1] = elem
            self.count += 1
            
    # Извлечение элемента с конца
    def popEnd(self):
        if len(self.arr) - (self.count - 1) <= len(self.arr):
            Deque._move(self, 'right')
            elem = self.arr[len(self.arr) - self.count] 
            self.arr[len(self.arr) - self.count] = 'empty'
            self.count -= 1
            return elem

### Задание 1 
Отсортировать строки файла, содержащие названия книг, в алфавитном порядке с
использованием 2-ух `Deque`.

In [180]:
books = []

# Функция для удаление символов переноса строки
def nlDel(s):
    pos = s.find('\n')
    if pos > -1:
        return s[0:pos]
    else:
        return s

# Получаем список книг и записываем его в лист Books 
f = open('task1/input.txt', encoding="utf8")
for line in f:
    books.append(nlDel(line))
f.close()
    
# Создаем две деки, с размером равным кол-ву книг
d1 = Deque(len(books))
d2 = Deque(len(books))

# Заполняем первую Деку книгами из списка
for book in range(len(books)):
    d1.push(books[book])

def sort(fr, to): 
    # Перекидываем первый элемент из одной деки в другую
    book = fr.pop()
    lastBook = book
    to.push(book)
    
    # Сортируем оставшие элементы, те у которых ,буквы на позиции j весит больше,
    # чем у предыдущего названия, идут в конец, иначе в начало
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = fr.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) < ord(book[j]):
                    to.pushEnd(book)
                if ord(lastBook[j]) > ord(book[j]):
                    to.push(book)
                lastBook = book
                break
            else:
                fr.push(book)
                
    # Проверяем, что все элементы отсортированы
    isSorted = True
    book = to.pop()
    lastBook = book
    fr.push(book)
    
    for i in range(len(books)-1):
        for j in range(min(len(lastBook),len(book))):
            book = to.pop()
            if ord(lastBook[j]) != ord(book[j]):
                if ord(lastBook[j]) > ord(book[j]):
                    isSorted = False
                fr.pushEnd(book)
                lastBook = book
                break
            else:
                to.push(book)
        
    if (not isSorted):
        return sort(fr,to)
    
# Сортируем список книг
sort(d1,d2)

# Открываем файл для записи 
f = open('task1/output.txt', 'w', encoding="utf8")
# Заносим отсортированные книги в файл
for i in range(d1.count):
    print(d1.pop(), file=f)
f.close()

### Задание 2
`Deque` содержит последовательность символов для шифровки сообщений. Дан
текстовый файл, содержащий зашифрованное сообщение. Пользуясь `Deque` нужном,
расшифровать текст. **Известно, что при шифровке каждый символ сообщения
заменялся следующим за ним в `Deque` по часовой стрелке через один.**


In [181]:
# Библиотека, которую использую для создания псевдослучайных комбинаций
import random

In [182]:
# Создаем словарь заменяемых символов и перемешиваем список этих символов
alphabet = list('АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя-.?!)(,:\\\'"')
random.shuffle(alphabet)
alphabet = ''.join(alphabet)

# Заполням дек
d = Deque(len(alphabet))
[d.push(alphabet[i]) for i in range(len(alphabet))]

# Строка для шифрования
s = 'Привет, как твои дела?'

# Функция для шифрования
def encode(t,d):
    t = list(t)
    # Прохожим все символы в исходной строке
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            # Если указанный символ есть, в нашем списке символов, то
            # заменяем символ из строки, след. через 1 символ из списка
            if c == t[i]:
                d.pushEnd(c)
                d.pushEnd(d.pop())
                c = d.pop()
                t[i] = c
            d.pushEnd(c)
    return ''.join(t)
    
# Функция для расшифровки    
def decode(t,d):
    t = list(t)
    # Прохожим все символы в исходной строке
    for i in range(len(t)):
        for j in range(d.count):
            c = d.pop()
            # Если указанный символ есть, в нашем списке символов, то
            # заменяем символ из строки, след. через 1 символ из списка (в обратном порядке)
            if c == t[i]:
                d.push(c)
                d.push(d.popEnd())
                c = d.popEnd()
                t[i] = c
            d.push(c)
    return ''.join(t)

print(encode(s,d), "- Зашифровонная фраза",)
print(decode(s,d), "- Расшифровка")

СрюаыкЬ ЛсЛ каХю Ты!сИ - Зашифровонная фраза
Привет, как твои дела? - Расшифровка


### Задание 3
Даны три стержня и n дисков различного размера. Диски можно надевать на
стержни, образуя из них башни. Перенести n дисков со стержня А на стержень С,
сохранив их первоначальный порядок. 

При переносе дисков необходимо соблюдать
следующие правила:
- на каждом шаге со стержня на стержень переносить только один диск;
- диск нельзя помещать на диск меньшего размера;
- для промежуточного хранения можно использовать стержень В.

Реализовать алгоритм, используя три стека вместо стержней А, В, С. Информация
о дисках хранится в исходном файле.


![Tower_of_Hanoi_4](https://upload.wikimedia.org/wikipedia/commons/6/60/Tower_of_Hanoi_4.gif "Решение такой задачи с 4 дисками")

In [183]:
disks = 3
# Создаем три стека для стержней, равные кол-ву дисков
A = Stack(disks) 
B = Stack(disks) 
C = Stack(disks) 

# Заполняем первый стержень всеми дисками
for i in range(disks, 0, -1):
    A.push(i)
    
print('A')
A.print()

print('--- Перемещения ---')

# Основная функция, с помощью который размещяем диски на стержнях
def move(st1, st2, let1, let2):
    if st1.count == 0 and st2.count > 0:
        st1.push(st2.pop())
    elif st1.count > 0 and st2.count == 0:
        st2.push(st1.pop())
    elif st1.top() > st2.top():
        st1.push(st2.pop())
    else:
        st2.push(st1.pop())
    
    print('\n',let1)
    st1.print()
    print(let2)
    st2.print()

if disks % 2 == 0:
    while C.count != disks:
        move(A, B, 'A', 'B')
        move(A, C, 'A', 'C')
        move(B, C, 'B', 'C')
else:
    while C.count != disks:
        move(A, C, 'A', 'C')
        if C.count == disks:
            break
        move(A, B, 'A', 'B')
        move(B, C, 'B', 'C')

print('--- Итог ---')
print('C')
C.print()

A
Stack = [3, 2, 1] len = 3
--- Перемещения ---

 A
Stack = [3, 2, 'empty'] len = 2
C
Stack = [1, 'empty', 'empty'] len = 1

 A
Stack = [3, 'empty', 'empty'] len = 1
B
Stack = [2, 'empty', 'empty'] len = 1

 B
Stack = [2, 1, 'empty'] len = 2
C
Stack = ['empty', 'empty', 'empty'] len = 0

 A
Stack = ['empty', 'empty', 'empty'] len = 0
C
Stack = [3, 'empty', 'empty'] len = 1

 A
Stack = [1, 'empty', 'empty'] len = 1
B
Stack = [2, 'empty', 'empty'] len = 1

 B
Stack = ['empty', 'empty', 'empty'] len = 0
C
Stack = [3, 2, 'empty'] len = 2

 A
Stack = ['empty', 'empty', 'empty'] len = 0
C
Stack = [3, 2, 1] len = 3
--- Итог ---
C
Stack = [3, 2, 1] len = 3


### Задание 4
Дан текстовый файл с программой на алгоритмическом языке. За один просмотр
файла проверить баланс круглых скобок в тексте, используя `Stack`.

In [184]:
# Основная функция
def isBracketsBalance(s):
    # Если число скобок нечетное сразу возвращяем результат
    if len(s) % 2 != 0:
        return False
    
    # Создаем Stack размером, со исходную строку
    # будем харнить в нем все открывабщиеся скобки.
    st = Stack(len(s))
    
    # Начинаем перебирать строку со скобками
    for c in s:
        if c == '(':
            st.push(c)
        # Если встретили закрывающую скобку, то проверяем не пуст ли Stack 
        # и если это не так, то удаляем скобку в начале списка
        if c == ')':
            if st.isEmpty():
                return False
            st.pop()
            
    # Возващяем boolean значение пуст ли Stack
    return st.isEmpty()

print(isBracketsBalance('()('))
print(isBracketsBalance('()())((())(()(())()'))
print(isBracketsBalance('(()())()()()()(()(()(())()))'))
print(isBracketsBalance('))(('))

False
False
True
False


### Задание 5
Дан текстовый файл с программой на алгоритмическом языке. За один просмотр
файла проверить баланс квадратных скобок в тексте, используя `Deque`.

In [185]:
# Основная функция
def isBracketsBalance(s):
    # Если число скобок нечетное сразу возвращяем результат
    if len(s) % 2 != 0:
        return False
    
    # Создаем Deque размером, со исходную строку
    # будем харнить в нем все открывабщиеся скобки.
    d = Deque(len(s))
    
    # Начинаем перебирать строку со скобками
    for c in s:
        if c == '[':
            d.push(c)
        # Если встретили закрывающую скобку, то проверяем не пуст ли Deque 
        # и если это не так, то удаляем скобку в начале списка
        if c == ']':
            if d.isEmpty():
                return False
            d.pop()
            
    # Возващяем boolean значение пуст ли Deque
    return d.isEmpty()

print(isBracketsBalance('[[]'))
print(isBracketsBalance('[][[][[[[]][[]]]]]]'))
print(isBracketsBalance('[[][][][[][]][][[][][][][]]][]'))
print(isBracketsBalance('][][[]'))

False
False
True
False


### Задание 6
Дан файл из символов. Используя `Stack`, за один просмотр файла напечатать
сначала все цифры, затем все буквы, и, наконец, все остальные символы, сохраняя
исходный порядок в каждой группе символов.

In [186]:
t = ''

symbs = ''
nums = ''
lets = ''

# Записываем весь текст из файла в переменную
f = open('task6/text.txt', encoding="utf8")
for line in f:
    t += line
f.close()

# Создаем стек и заполняем его всеми символами подряд из файла в обратном порядке
st = Stack(len(t))
for c in reversed(t):
    st.push(c)

# Проходим по всему стеку ОДИН РАЗ, и сравниваем коды символов с необходимыми, 
# помещяем символы в соответствующие массивы
for i in range(len(t)):
    # Если символ это цифра
    if ord(st.top()) >= 48 and ord(st.top()) < 58:
        nums += st.pop()
    # Если это спец. символ
    elif ord(st.top()) >= 33 and ord(st.top()) < 48 or ord(st.top()) >= 58 and ord(st.top()) < 65:
        symbs += st.pop()
    # Если это пробел
    elif ord(st.top()) == 32:
        if len(symbs)-1 > -1 and ord(symbs[len(symbs)-1]) != 32:
            symbs += ' '
        if len(nums)-1 > -1 and ord(nums[len(nums)-1]) != 32:
            nums += ' '
        if len(lets)-1 > -1 and ord(lets[len(lets)-1]) != 32:
            lets += ' '
        st.pop()
    # Все остальное
    else:
        lets += st.pop()
        
print(nums)
print(lets)
print(symbs)

1994 95 95 1995 3 3 81 2002 9 2002 2004 30 90 2005 
Изначально нацеленный на разработку видеоигр DirectX стал популярен и в других областях разработки программного обеспечения К примеру DirectX наряду с OpenGL получил очень широкое распространение в инженерномматематическом ПО В году Microsoft была практически готова выпустить следующую революционную версию Windows — Windows Главным фактором определяющим насколько популярна будет новая ОС являлся набор программ которые можно будет запускать под её управлением В компании пришли к выводу что пока разработчики видят DOS более подходящей для написания игровых приложений коммерческий успех новой ОС весьма сомнителен DOS позволяла разработчику получить прямой доступ к видеокарте клавиатуремышиджойстику и прочим частям системы в то время как Windows с её защищённой моделью памяти предоставляла более стандартизованный но в то же время весьма ограниченный и накладный доступ к устройствам Microsoft нуждалась в новом способе дать разработчику всё

### Задание 8
Дан текстовый файл. Используя `Stack`, сформировать новый текстовый файл,
содержащий строки исходного файла, записанные в обратном порядке: первая
строка становится последней, вторая – предпоследней и т.д.

In [187]:
# Функция для удаление символов переноса строки
def nlDel(s):
    pos = s.find('\n')
    if pos > -1:
        return s[0:pos]
    else:
        return s

books = []
    
# Открываем файл для записи
f = open('task1/output.txt', encoding="utf8")
# Копируем книги из файла в список "Books"
for line in f:
    books.append(nlDel(line))
f.close()

s = Stack(len(books))

# Заполняем Stack
for i in range(len(books)):
    s.push(books[i])

# Открываем файл для записи 
f = open('task8/output-reverse.txt', 'w', encoding="utf8")
# Заносим отсортированные книги в файл
for i in range(s.count):
    print(s.pop(), file=f)
f.close()